In [1]:
from pathlib import Path

import dash
import dash_core_components as dcc
from dash.dependencies import Input, Output
import dash_html_components as html
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pd
import plotly.graph_objects as go

### Load the data

In [2]:
DATA_ROOT = Path('..') / 'data'

In [3]:
dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])

sensor_df.to_csv('sensor_df.csv')

### Make data for plotting

In [4]:
# We join the participant name with the activity label of the row
# to get ['d1p01M-bed', 'd1p01M-bed', ...] etc.
values = sensor_df['participant'].str.cat(sensor_df['activity_label'], sep='-')

# Using shift to compare consecutive values we can obtain a boolean series
# which is `True` at the start of every activity, and `False` otherwise.
activity_start = values.shift(1) != values

# The cumsum trick converts the boolean values to integers,
# so `True` becomes `1` and `False` becomes `0`. 
# As `activity_start` is `True` at the start of every activity and `False`
# otherwise, successive activities in the data get an different (increasing)
# number:
#
#    list(pd.Series([True, False, True, True]).cumsum()) # <- [1, 1, 2, 3]
#
sensor_df['activity_block'] = activity_start.cumsum()


In [5]:
# Define some colors
SENSOR_COLORS = {
  'front': '#1f77b4',
  'vertical': '#ff7f0e',
  'lateral': '#2ca02c',
}

ACTIVITY_COLORS = {
  4: '#fed9a6',
  1: '#b3cde3',
  2: '#ccebc5',
  3: '#decbe4',
}

BUTTON_COLORS = {
    'fg': '#444444',
    'bg': '#DDDDDD'
}

# And a button CSS style
BUTTON_STYLE = dict(
    outline='none !important',
    backgroundColor=BUTTON_COLORS['bg'],
    color=BUTTON_COLORS['fg'],
    minWidth='7.5rem',
    margin='0.5rem',
    padding='0.5rem',
    border='none',
    borderRadius='3px'
)

### Plot the data

In [6]:
app = JupyterDash(__name__)


# Define the layout of the app.
# Requires a bit of HTML and CSS knowledge - but but _too_ much :)
app.layout = html.Div(children=[
    html.Div(children=[
        dcc.Graph(id='plot'),
    ]),
    html.Div(children=[
        html.Div(children=[
            html.Button(
                'Previous participant', 
                id='prev-participant', 
                n_clicks=0, 
                style=BUTTON_STYLE),
            html.Button(
                'Next participant', 
                id='next-participant', 
                n_clicks=0, 
                style=BUTTON_STYLE),
        ]),
        html.Div(children=[
            html.Div(children=[
                html.Label('Smoothing'),
            ]),
            html.Div(children=[
                dcc.Slider(
                    id='smoothing-slider', 
                    min=0, 
                    max=10, 
                    value=0, 
                    marks={str(m):'' for m in range(11)}, step=None)
            ], style=dict(minWidth='256px', maxHeight='10px'))
        ], style=dict(display='flex', paddingLeft='1rem'))
    ],
    style=dict(display='flex', justifyContent='center', width='100%', alignItems='center'))
])

# The callback responds to changes in its inputs
# (i.e., the buttons and slider) and outputs a new figure
# on the basis of these values.
@app.callback(
    Output('plot', 'figure'),
    [
        Input('next-participant', 'n_clicks'), 
        Input('prev-participant', 'n_clicks'),
        Input('smoothing-slider', 'value')]
)
def update_figure(next_button_clicks, prev_button_clicks, smoothing):
    """When something changes (e.g., button click or smoothing change) update the figure"""
    
    participants = sensor_df['participant'].unique()
    
    # Only have the number of time the buttons have been clicked
    # (this is all I understand for now) so mess around to get
    # the index of the participant.
    participant_index = (next_button_clicks - prev_button_clicks) % len(participants)
    while participant_index < 0:
        participant_index += len(participants)
    participant = participants[participant_index]
    
    # Subset the data
    df = sensor_df[sensor_df['participant'] == participant]
    
    xmin, xmax = df['time'].min(), df['time'].max()
    nactivities = df['activity_block'].nunique()
    activity_labels = []
    
    # Plot the figure.
    fig = go.Figure()
    for index, group in enumerate(df.groupby('activity_block')):
        _, activity_df = group
        start = activity_df['time'].min()
        end = activity_df['time'].max()
        
        # Plot the activity rectangles.
        fig.add_trace(go.Scatter(
            x=[start, end, end, start],
            y=[-2, -2, 2, 2],
            line_color=ACTIVITY_COLORS[activity_df['activity'].values[0]],
            line_width=0,
            fill='toself',
            mode='lines',
            showlegend=False,
            name='',
        ))
        
        # Plot the sensor data.
        for sensor in ['front', 'vertical', 'lateral']:
            original = activity_df[sensor]
            
            # Smooth if required using moving average.
            if smoothing:
                smoothed = []
                for i in range(len(original)):
                    j = max(i - smoothing, 0)
                    k = min(i + smoothing + 1, len(original))
                    frame = original[j:k]
                    smoothed.append(sum(frame) / len(frame))
                    
                # Show origin values
                fig.add_trace(go.Scatter(
                    x=activity_df['time'],
                    y=original,
                    line_color=SENSOR_COLORS[sensor],
                    mode='lines',
                    opacity=0.333,
                    showlegend=False,
                    name='',
            ))
            else:
                smoothed = original
            
            # Plot the points
            fig.add_trace(go.Scatter(
                x=activity_df['time'],
                y=smoothed,
                line_color=SENSOR_COLORS[sensor],
                mode='lines+markers',
                marker_size=5,
                showlegend=False,
                name='',
            ))
            
        # Need to store the positions and text of the activity labels
        # to ensure they are drawn after everything else otherwise
        # subsequent activity rectangles will be drawn over the labels.
        x = (start + end) / 2
        y = -2 + (index + 0.5) / nactivities * 4
        activity_labels.append((x, y, activity_df['activity_label'].values[0]))
    
    # Done plotting, so draw activity labels.
    for x, y, text in activity_labels:
        fig.add_annotation(
        x=x, 
        y=y, 
        xref='x', # use axis coordinates
        yref='y', # use axis coordinates
        showarrow=False, 
        text=text)
    
    # Show the participant label
    fig.add_annotation(
        x=(xmin + xmax) / 2, 
        y=-2, 
        xref='x', 
        yref='y', 
        yanchor='bottom',
        showarrow=False, 
        text=f'{participant} {participant_index + 1}/{len(participants)}')
    
    # Set the margins 
    fig.update_layout(
        margin=dict(l=24, r=24, t=32, b=32)
    )
    
    # Figure is complete, so return it and let Dash to everything else
    return fig

app.run_server(debug=True, mode='inline')